In [ ]:
from functions import sample_uniform_parameters, sample_correlated_parameters, output_from_parameters, output_from_parameters_with_noise, obtain_theta, perform_CNLS_LASSO, obtain_beta_unique, add_random_variables, add_random_correlated_variables, delete_random_variables, retrieve_results, retrieve_results_regular_lasso
from pystoned.plot import plot2d
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import copy

In [ ]:
# CORRELATION FOR TRUE INPUTS
# Fill in the desired correlations between the variables in the dictionary corrs
nr_of_corr_variables = 9

corrs_TRUE = {i: {} for i in range(0, nr_of_corr_variables+1)}
corrs_FALSE = {i: {} for i in range(0, nr_of_corr_variables+1)}

corrs_TRUE[1][2] = 0.8

# CORRELATION FOR FALSE INPUTS
corrs_FALSE[1][2] = 0.8
corrs_FALSE[1][3] = 0.8
corrs_FALSE[2][3] = 0.8
corrs_FALSE[4][5] = 0.8
corrs_FALSE[6][7] = 0.8


# Simulation for 25 DMUS

In [ ]:
import multiprocessing
import functions
from itertools import repeat
import pandas as pd


# Specify the input parameters
CORRELATION = CORRELATION_REDUNDANT_VARIABLES = True
TRUE_INPUTS = 2
REDUNDANT_INPUTS = 7
NR_DMU = 25
ETA = 1
VAR_mu = 0.1
REPITITIONS = 500
EMAIL = 'maxklaasbakker@gmail.com'

pool = multiprocessing.Pool()

# Loop through different combination values of var_mu and ETA
for VAR_mu, ETA in zip([0.1, 0.7],[0.01,0.5]):
    result = pool.starmap(functions.run_simulation_CNLS_LASSO, 
        zip(range(REPITITIONS),
        repeat(CORRELATION), repeat(CORRELATION_REDUNDANT_VARIABLES),
        repeat(TRUE_INPUTS), repeat(REDUNDANT_INPUTS), repeat(NR_DMU), 
        repeat(ETA), repeat(VAR_mu),
        repeat(corrs_TRUE), repeat(corrs_FALSE), repeat(EMAIL))) 
    result_df = pd.DataFrame(result)
    result_df.to_csv(
        f"results_SCNLS_LASSO_2_true_7_false_25_dmu_not_sorted_{str(VAR_mu).replace('.', '_')}_var_eta_{str(ETA).replace('.', '_')}_CORR_{CORRELATION}.csv", sep=';')

# Simulation for 100 DMUs

In [ ]:
import multiprocessing
import functions
from itertools import repeat


CORRELATION = True
CORRELATION_REDUNDANT_VARIABLES = True
TRUE_INPUTS = 2
REDUNDANT_INPUTS = 7
NR_DMU = 100
ETA = 1
VAR_mu = 0.1
eta_reg_LASSO = 0.01
REPITITIONS = 30
EMAIL = 'maxklaasbakker@gmail.com'

nr_of_corr_variables = 9

corrs_TRUE = {i: {} for i in range(0, nr_of_corr_variables+1)}
corrs_FALSE = {i: {} for i in range(0, nr_of_corr_variables+1)}

corrs_TRUE[1][2] = 0.8

# CORRELATION FOR FALSE INPUTS
corrs_FALSE[1][2] = 0.8
corrs_FALSE[1][3] = 0.8
corrs_FALSE[2][3] = 0.8
corrs_FALSE[4][5] = 0.8
corrs_FALSE[6][7] = 0.8

# eerste 5  in var=0.7 met eta 0.3 ; var=0.1 met eta 0.01
# 6 t/m 9 in var=0.7 met eta 0.5 ; var=0.1 met eta 0.01

for VAR_mu in [0.1, 0.7]:
    results_SCNLS_LASSO_run = {}
    results_random_run = {}
    results_reg_LASSO_run = {}
    for i in range(1,10):
        print(i)
        if i >2:
            TRUE_INPUTS = 2
            REDUNDANT_INPUTS = i-2
        else:
            TRUE_INPUTS = i
            REDUNDANT_INPUTS = 0
        
        if VAR_mu == 0.1:
            ETA = 0.01
        elif i<=5 & VAR_mu == 0.7:
            ETA = 0.3
        elif i>5 & VAR_mu == 0.7:
            ETA = 0.5

        
        pool = multiprocessing.Pool()
        result = pool.starmap(functions.run_simulation_CNLS_LASSO_RANDOM, 
            zip(range(REPITITIONS),
            repeat(CORRELATION), repeat(CORRELATION_REDUNDANT_VARIABLES),
            repeat(TRUE_INPUTS), repeat(REDUNDANT_INPUTS), repeat(NR_DMU), 
            repeat(ETA), repeat(VAR_mu),
            repeat(corrs_TRUE), repeat(corrs_FALSE), repeat(eta_reg_LASSO), repeat(EMAIL)))
        result = pd.DataFrame(result).to_dict()
        results_SCNLS_LASSO_run[i] = pd.DataFrame(result[0]).T
        results_random_run[i] = pd.DataFrame(result[1]).T
        results_reg_LASSO_run[i] = pd.DataFrame(result[2]).T

    results_SCNLS_df =pd.DataFrame()
    results_random_df =pd.DataFrame()
    results_reg_LASSO_df =pd.DataFrame()

    for i in range(1,10):

        results_SCNLS_df_i = results_SCNLS_LASSO_run[i].rename(columns = lambda x: x+f'_{i}')
        results_SCNLS_df = pd.concat([results_SCNLS_df, results_SCNLS_df_i], axis=1)
        results_random_df_i = results_random_run[i].rename(columns = lambda x: x+f'_{i}')
        results_random_df = pd.concat([results_random_df, results_random_df_i], axis=1)
        results_reg_LASSO_df_i = results_reg_LASSO_run[i].rename(columns = lambda x: x+f'_{i}')
        results_reg_LASSO_df = pd.concat([results_reg_LASSO_df, results_reg_LASSO_df_i], axis=1)

    results_SCNLS_df.to_csv(f"testresults_SCNLS_df_100_dmus_30_reps_eta_{str(ETA).replace('.', '_')}_notsorted_var_{str(VAR_mu).replace('.', '_')}_CORR_{CORRELATION}.csv")
    results_random_df.to_csv(f"testresults_random_df_100_dmus_30_reps_eta_{str(ETA).replace('.', '_')}_notsorted_var_{str(VAR_mu).replace('.', '_')}_CORR_{CORRELATION}.csv")
    results_reg_LASSO_df.to_csv(f"testresults_reg_LASSO_df_100_dmus_30_reps_eta_{str(ETA).replace('.', '_')}_notsorted_var_{str(VAR_mu).replace('.', '_')}_CORR_{CORRELATION}.csv")